In [7]:
import xml.etree.ElementTree as ET
import numpy as np
import glob
import os
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from six import BytesIO

2023-04-04 16:49:02.133533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#start label img
os.system("labelimg")

Image:/Users/ml/Desktop/E2E_project/object_detection/data/test.png -> Annotation:/Users/ml/Desktop/E2E_project/object_detection/data/test.xml


0

In [2]:
#xml stored in following directory
data_dir = ".//data"

In [3]:
#get all the xml file names generated after labelling from previous step
file_names = []
os.chdir(data_dir )
for file in glob.glob("*.xml"):
    file_names.append(file)

In [4]:
bbox_formatted_all = []
name_obj_all = []
for file_name in file_names:
    
    tree = ET.parse(file_name)
    
    root = tree.getroot()

    bbox_formatted= []
    name_obj_formatted= []
    obj_bbox = root.findall('./object')
    name_obj = []
    for item in obj_bbox:
        bbox = []
        name_obj.append(item.find('name').text)
        bbox_cord= item.find('./bndbox')
        bbox.append(
            [
            bbox_cord.find("ymin").text,
            bbox_cord.find("xmin").text,
            bbox_cord.find("ymax").text,
            bbox_cord.find("xmax").text
            ]  
        )
        bbox_formatted.append((bbox[0]))
    bbox_formatted_all.append(np.array(bbox_formatted, dtype = np.float32)/640.0) #640 is both and width dimention.
    name_obj_all.append(np.array(name_obj))


In [5]:
#bbox in npy format, this is saved in data file
bbox_fn = 'bbox.npy'
with open(bbox_fn, 'wb') as f:
    np.save(f, bbox_formatted_all)

In [6]:
#index of the class list
indices_class_fn = 'indices_class_list.npy'
name_obj_all = np.array(name_obj_all).astype(np.int0)
with open(indices_class_fn, 'wb') as f:
    np.save(f, name_obj_all)

In [ ]:
#utility - visulaization

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  #print("width, height",im_width,im_height)
  #print("image data",np.array(image.getdata()))
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)



In [ ]:
#visualizing bbox


#get all the png file names generated after labelling from previous step
image_file_names = []
os.chdir(data_dir )
for file in glob.glob("*.png"):
    image_file_names.append(file)

train_images_np = []
#for i in range(1):
for image_path in image_file_names:
  train_images_np.append(load_image_into_numpy_array(image_path))

gt_boxes = np.load(bbox_fn)
dummy_scores = np.array([1.0], dtype=np.float32)  # give boxes a score of 100%

plt.figure(figsize=(30, 15))
for idx in range(1):
  plt.subplot(2, 3, idx+1)
  dummy_scores = tf.convert_to_tensor(np.ones((1, gt_boxes[idx].shape[0]),dtype=np.float32))

  """CM start """
  class_id = tf.convert_to_tensor(np.ones((1,gt_boxes[idx].shape[0]), dtype=np.int32))
  indices = [[0,1]]
  updates = [2]
  class_id = tf.tensor_scatter_nd_update(class_id, indices, updates)
  """ CM end """

  tr_img_expdims = np.expand_dims(train_images_np[idx], axis=0)

  plot_detections_tensor(
      tf.convert_to_tensor(tr_img_expdims),
      tf.convert_to_tensor(np.expand_dims(gt_boxes[idx], axis=0)),
      #tf.convert_to_tensor(np.ones((1,gt_boxes[idx].shape[0]), dtype=np.int32)), #original
      class_id, #cm
      dummy_scores, category_index)
plt.show()